## Генерация выборок


In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

def gen_gaussian_lin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu, sigma, n)
    y = 0.6 * x + 0.4 * z
    return x, y

def gen_notgaussian_notlin_mono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu*2, sigma*10, n)
    y = 2*x**3 + 3*z
    return x, y

def gen_notgaussian_notlin_notmono(mu, sigma, n):
    x = np.random.normal(mu, sigma, n)
    z = np.random.normal(mu*2, sigma*2, n)
    y = x ** 2 + z ** 2
    return x, y

In [2]:
def make_test(test, count_view= 100, n =100):
    freq_error_gaussian = 0
    freq_error_notgaussian_liner = 0
    freq_error_notgaussian_notliner = 0
    
    for _ in range(n):
        x_gaus, y_gaus = gen_gaussian_lin_mono(2, 10, count_view)
        x_notgaus_liner, y_notgaus_liner = gen_notgaussian_notlin_mono(2, 10, count_view)
        x_notgaus_notliner, y_notgaus_notliner = gen_notgaussian_notlin_notmono(2, 10, count_view)
        freq_error_gaussian = freq_error_gaussian + test(x_gaus,y_gaus)[2]
        freq_error_notgaussian_liner =freq_error_notgaussian_liner + test(x_notgaus_liner,y_notgaus_liner)[2]
        freq_error_notgaussian_notliner =freq_error_notgaussian_notliner + test(x_notgaus_notliner,y_notgaus_notliner)[2]
    
    return freq_error_gaussian/n, freq_error_notgaussian_liner/n, freq_error_notgaussian_notliner/n
        
        
    

## Выборочный коэффициент корреляции

Проверка зависимости двух нормально распределённых признаков

In [3]:
def corr_test(x, y, alpha = 0.05):
    # флаг, который указывает, принимаем H0 или отвергаем : 1-отвергаем, 0 - принимаем
    complete = 0
    # кол-во элементов в выборке
    n = len(x)
    # вычисялем выборочное среднее выборок
    x_mean = x.mean()
    y_mean = y.mean()
    # вычисляем выборочный коэф корреляции
    r = (1 / n * np.sum((x - x_mean) * (y - y_mean))) / \
    (1 / n * np.sum((x - x_mean) ** 2) * 1 / n * np.sum((y - y_mean) ** 2)) ** 0.5
    t_value = n ** 0.5 * r
    p_value = np.min(np.array([2 * stats.norm.cdf(t_value), 2 - 2 * stats.norm.cdf(t_value)]), axis=0)
    if (p_value > alpha / 2) and (p_value < (1 - alpha / 2)):
        complete = 0
    else:
        complete = 1
    return r, t_value, p_value, complete

In [4]:
f_gaus, f_ngaus, g_ngaus_nl = make_test(corr_test)
print(f"""
    Частота ошибок критерия корреляции
    Гауссова {f_gaus}
    Не гауссова линеная {f_ngaus}
    Не гаусова не линейная {g_ngaus_nl}
""")


    Частота ошибок критерия корреляции
    Гауссова 1.0058620603103919e-15
    Не гауссова линеная 3.022937455909869e-13
    Не гаусова не линейная 0.39427462336984326



## Критерий Спирмана

Признаки ненормальны, подозревается монотонная зависимость

In [5]:
def ranging(x):
    x_sorted = np.sort(x)
    R = np.array([])
    elems = np.array([])
    x_rangs =  np.array([])
    i = 1
    while i <= len(x):
        x_count = (x_sorted == x_sorted[i-1]).sum()
        if x_count == 1:
            R = np.append(R, i)
            elems = np.append(elems, x_sorted[x_sorted == x_sorted[i-1]])
        else:
            for j in range(x_count):
                R = np.append(R, i + (c - 1) / 2)
                elems = np.append(elems, x_sorted[x_sorted == x_sorted[i-1]][0])
        i += x_count
    for i in range(len(x)):
        idx = np.where(elems == x[i])
        x_rangs = np.append(x_rangs, R[idx[0]])
    return x_rangs

def spearman(x, y, alpha = 0.05):
    complete = 0
    n = len(x)
    x_rangs = ranging(x)
    y_rangs = ranging(y)
    r_spearman = 1 - 6 / (n ** 3 - n) * np.sum((x_rangs - y_rangs) ** 2)
    t_value = (n-1) ** 0.5 * r_spearman
    p_value = np.min(np.array([2 * stats.norm.cdf(t_value), 2 - 2 * stats.norm.cdf(t_value)]), axis=0)
    if (p_value > alpha / 2) and (p_value < (1 - alpha / 2)):
        complete = 0
    else:
        complete = 1
    return r_spearman, t_value, p_value, complete

In [6]:
f_gaus, f_ngaus, g_ngaus_nl = make_test(spearman)
print(f"""
    Частота ошибок критерия Спирмена
    Гауссова {f_gaus}
    Не гауссова линеная {f_ngaus}
    Не гаусова не линейная {g_ngaus_nl}
""")


    Частота ошибок критерия Спирмена
    Гауссова 5.246247880563715e-14
    Не гауссова линеная 0.0
    Не гаусова не линейная 0.35563447801335957



## Критерий Хи-квадрат Пирсона

In [7]:
def to_nominal(x, k):
    x_sort = np.sort(x)
    x_in_scale = np.zeros(k)
    slices = np.linspace(x_sort[0], x_sort[-1], k+1)
    for i in range(k):
        x_part = x[x >= slices[i]]
        x_count = (x_part <= slices[i+1]).sum()
        x_in_scale[i] = x_count
    return x_in_scale

def pirson(x, y, alpha = 0.05):
    complete = 0
    n = len(x)
    x_nom = to_nominal(x, 5)
    y_nom = to_nominal(y, 5)
    matrix = np.concatenate(([x_nom], [y_nom])) 
    chi2_value = 0
    for i in range(2):
        for j in range(len(x_nom)):
            chi2_value += (matrix[i][j] - (np.sum(matrix[i]) * np.sum(matrix[:,j]) / np.sum(matrix))) ** 2 \
            / (np.sum(matrix[i]) * np.sum(matrix[:,j]) / np.sum(matrix))
            
   
    p_value = 1 - stats.chi2.cdf(chi2_value, (len(x_nom) - 1))
    if p_value > alpha:
        complete = 0
    else:
        complete = 1
    return chi2_value, p_value, complete

In [8]:
f_gaus, f_ngaus, g_ngaus_nl = make_test(pirson)
print(f"""
    Частота ошибок критерия Спирмена
    Гауссова {f_gaus}
    Не гауссова линеная {f_ngaus}
    Не гаусова не линейная {g_ngaus_nl}
""")


    Частота ошибок критерия Спирмена
    Гауссова 0.21
    Не гауссова линеная 0.99
    Не гаусова не линейная 1.0

